# **Generación del dataset de Nivel 3**

En este notebook generamos el dataset del Nivel 3, en este caso utilizando el LLM V1 de DeepSeek. Por seguridad hemos quitado la clave de la API.

Me da siempre la sensación que DeepSeek, cuando le preguntas por código, tiende a dar soluciones demasiado complejas, aunque al menos parecen funcionar.

Cargamos las librerías, que fortunadamente no es necesario instalar nada nuevo.

In [ ]:
import pandas as pd
import numpy as np
import json
import random
import time
from typing import List, Dict, Optional, Tuple
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
from datetime import datetime
import pickle
import os

Cargamos el dataset original, y lo inspeccionamos brevemente.

In [ ]:
strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str', 'cod_sec' : 'str'}

In [ ]:
df_eleccion = pd.read_csv('/content/drive/MyDrive/Practica_LLM_Engineering_25/df_elecciones_19.csv', dtype = strings)

In [ ]:
df_eleccion.head()

,Elecciones,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Distrito,...,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015
0,Abril 2019,022019041010400101001,01,04,04001,0400101001,Andalucía,Almería,Abla,1,...,9159.0,8788.0,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0
1,Abril 2019,022019041010400201001,01,04,04002,0400201001,Andalucía,Almería,Abrucena,1,...,8827.0,8107.0,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0
2,Abril 2019,022019041010400301001,01,04,04003,0400301001,Andalucía,Almería,Adra,1,...,8965.0,8267.0,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0
3,Abril 2019,022019041010400301002,01,04,04003,0400301002,Andalucía,Almería,Adra,1,...,8599.0,7941.0,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0
4,Abril 2019,022019041010400301003,01,04,04003,0400301003,Andalucía,Almería,Adra,1,...,8076.0,7150.0,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0


In [ ]:
df_eleccion.columns

Index(['Elecciones', 'Sección', 'cod_ccaa', 'cod_prov', 'cod_mun', 'cod_sec',
       'CCAA', 'Provincia', 'Municipio', 'Distrito', 'Censo_Esc',
       'Votos_Total', 'Participación', 'Nulos', 'Votos_Válidos', 'Blanco',
       'V_Cand', 'PP', 'PSOE', 'Cs', 'UP', 'IU', 'VOX', 'UPyD', 'MP', 'CiU',
       'ERC', 'JxC', 'CUP', 'DiL', 'PNV', 'Bildu', 'Amaiur', 'CC', 'FA', 'TE',
       'BNG', 'PRC', 'GBai', 'Compromis', 'PACMA', 'Otros', '% PP', '% PSOE',
       '% UP', '% VOX', '% Cs', '% IU', 'Ganador', 'Segundo', 'Tercero',
       'Cuarto', 'Quinto', 'edad_0-4', 'edad_5-9', 'edad_10-14', 'edad_15-19',
       'edad_20-24', 'edad_25-29', 'edad_30-34', 'edad_35-39', 'edad_40-44',
       'edad_45-49', 'edad_50-54', 'edad_55-59', 'edad_60-64', 'edad_65-69',
       'edad_70-74', 'edad_75-79', 'edad_80-84', 'edad_85-89', 'edad_90-94',
       'edad_95-99', 'edad_100 y más', 'Población Total', 'Hombres', 'Mujeres',
       '% mayores 65 años', '% 20-64 años', '% menores 19 años',
       'Afiliados S

Activamos el logging.

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

Vamos con la clase que genera las preguntas y resupuestas más complejas, las del Nivel 3. Utilizamos el LLM V1 de DeepSeek para generar una pregunta compleja en base a un prompt inicial, que tiene 4 versiones: correlación avanzada, impacto socioeconómico, tendencia temporal o 'auto' (pregunta especialmente compleja).

La clase también consigue la respuesta del LLM, naturalmente, que parsea a continuación.

Como le pregunté a DeepSeek por el coste, incorpora una estimación de lo que voy gastando, que es casi nada.

In [ ]:
class DeepSeekAPIGenerator:
    def __init__(self, api_key: str):
        """
        Inicializa el generador con la API Key de DeepSeek

        Args:
            api_key: Tu API Key de DeepSeek
        """
        self.api_key = api_key
        self.base_url = "https://api.deepseek.com/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }

        # Estadísticas de uso
        self.total_tokens = 0
        self.total_cost = 0.0
        self.requests_count = 0

        # Rate limiting
        self.rate_limit_delay = 0.5  # segundos entre requests
        self.last_request_time = 0

    def _calculate_cost(self, prompt_tokens: int, completion_tokens: int) -> float:
        """
        Calcula costo según pricing de DeepSeek-V2
        (Actualiza según pricing oficial)
        """
        # Precios aproximados (verifica en deepseek.com)
        input_cost_per_1k = 0.00014  # $0.14 por 1M tokens input
        output_cost_per_1k = 0.00028  # $0.28 por 1M tokens output

        cost = (prompt_tokens / 1000) * input_cost_per_1k
        cost += (completion_tokens / 1000) * output_cost_per_1k

        return cost

    def _wait_for_rate_limit(self):
        """Control de rate limiting"""
        elapsed = time.time() - self.last_request_time
        if elapsed < self.rate_limit_delay:
            time.sleep(self.rate_limit_delay - elapsed)
        self.last_request_time = time.time()

    def generate_complex_qa(self, data_sample: pd.DataFrame,
                           sample_size: int = 5,
                           question_type: str = "auto") -> Optional[Dict]:
        """
        Genera una pregunta-respuesta compleja basada en datos reales

        Args:
            data_sample: DataFrame con datos de muestra
            sample_size: Número de filas a incluir en el prompt
            question_type: Tipo de pregunta a generar

        Returns:
            Diccionario con pregunta, respuesta y metadatos
        """
        try:
            # 1. Preparar muestra de datos para el prompt
            data_summary = self._prepare_data_summary(data_sample, sample_size)

            # 2. Crear prompt optimizado
            prompt = self._create_optimized_prompt(data_summary, question_type)

            # 3. Llamar a la API
            self._wait_for_rate_limit()

            payload = {
                "model": "deepseek-chat",  # o "deepseek-v2" si disponible
                "messages": [
                    {
                        "role": "system",
                        "content": """Eres DeepSeek, un experto en análisis de datos electorales y socioeconómicos
                        con especialización en España. Tu tarea es crear preguntas analíticas complejas y sus
                        respuestas basadas en datos reales.

                        REQUISITOS:
                        1. Las preguntas deben combinar múltiples dimensiones (geografía, tiempo, partidos, variables socioeconómicas)
                        2. Las respuestas deben incluir cálculos específicos, interpretaciones y contexto
                        3. Usar solo los datos proporcionados, no inventar información
                        4. Formato estricto: 'P: [pregunta]' y 'R: [respuesta]' en líneas separadas
                        """
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "temperature": 0.7,
                "max_tokens": 1500,
                "top_p": 0.9
            }

            response = requests.post(
                self.base_url,
                headers=self.headers,
                json=payload,
                timeout=60
            )

            if response.status_code != 200:
                logger.error(f"Error API: {response.status_code} - {response.text}")
                return None

            result = response.json()

            # 4. Actualizar estadísticas de costo
            usage = result.get("usage", {})
            prompt_tokens = usage.get("prompt_tokens", 0)
            completion_tokens = usage.get("completion_tokens", 0)

            cost = self._calculate_cost(prompt_tokens, completion_tokens)
            self.total_tokens += prompt_tokens + completion_tokens
            self.total_cost += cost
            self.requests_count += 1

            # 5. Parsear respuesta
            content = result["choices"][0]["message"]["content"]
            qa_dict = self._parse_qa_response(content)

            if qa_dict:
                # Añadir metadatos
                qa_dict["metadata"] = {
                    "tipo": "nivel3",
                    "subtipo": question_type,
                    "muestra_tamano": sample_size,
                    "tokens_utilizados": prompt_tokens + completion_tokens,
                    "costo_estimado": cost,
                    "timestamp": datetime.now().isoformat()
                }

                logger.info(f"Generado QA #{self.requests_count} | Costo acumulado: ${self.total_cost:.4f}")
                return qa_dict

            return None

        except Exception as e:
            logger.error(f"Error generando QA: {str(e)}")
            return None

    def _prepare_data_summary(self, data: pd.DataFrame, sample_size: int) -> str:
        """
        Prepara un resumen estructurado de los datos para el prompt
        Minimiza tokens manteniendo información relevante
        """
        if len(data) > sample_size:
            data = data.sample(sample_size, random_state=42)

        summary_lines = []

        # Columnas clave a incluir
        key_columns = [
            'Municipio', 'Provincia', 'CCAA',
            '% PP', '% PSOE', '% VOX', '% Cs', '% UP',
            'Participación', 'Censo_Esc', 'Votos_Total',
            'Renta persona 2017', 'Renta hogar 2017',
            '% mayores 65 años', '% 20-64 años'
        ]

        available_columns = [col for col in key_columns if col in data.columns]

        for idx, row in data.iterrows():
            line_parts = []

            # Información geográfica
            if 'Municipio' in data.columns:
                line_parts.append(f"Mun: {row['Municipio']}")
            if 'Provincia' in data.columns:
                line_parts.append(f"Prov: {row['Provincia']}")

            # Datos electorales
            electoral_data = []
            for col in ['% PP', '% PSOE', '% VOX', '% Cs', '% UP', 'Participación']:
                if col in data.columns and pd.notna(row[col]):
                    electoral_data.append(f"{col}: {row[col]:.1f}")

            if electoral_data:
                line_parts.append("Elec: " + ", ".join(electoral_data))

            # Datos socioeconómicos
            socio_data = []
            for col in ['Renta persona 2017', 'Renta hogar 2017', '% mayores 65 años']:
                if col in data.columns and pd.notna(row[col]):
                    socio_data.append(f"{col}: {row[col]:.0f}")

            if socio_data:
                line_parts.append("Socio: " + ", ".join(socio_data))

            summary_lines.append(" | ".join(line_parts))

        # Añadir estadísticas agregadas
        stats_lines = ["\nESTADÍSTICAS AGREGADAS:"]

        for col in ['% PP', '% PSOE', 'Participación', 'Renta persona 2017']:
            if col in data.columns:
                mean_val = data[col].mean()
                std_val = data[col].std()
                stats_lines.append(f"{col}: μ={mean_val:.1f}, σ={std_val:.1f}")

        return "DATOS DE MUESTRA:\n" + "\n".join(summary_lines) + "\n" + "\n".join(stats_lines)

    def _create_optimized_prompt(self, data_summary: str, question_type: str) -> str:
        """
        Crea prompts optimizados para diferentes tipos de preguntas complejas
        """
        prompts = {
            "correlacion_avanzada": f"""{data_summary}

BASÁNDOTE EXCLUSIVAMENTE EN LOS DATOS PROPORCIONADOS, genera:

1. UNA PREGUNTA COMPLEJA que analice la relación entre al menos TRES variables diferentes
   (ej: voto, datos socioeconómicos y geografía). La pregunta debe requerir razonamiento estadístico.

2. UNA RESPUESTA DETALLADA que incluya:
   - Cálculos específicos con números reales de los datos
   - Interpretación de patrones encontrados
   - Contexto político/social relevante
   - Limitaciones del análisis

FORMATO ESTRICTO (sin desviaciones):
P: [tu pregunta aquí]
R: [tu respuesta aquí]

La pregunta debe ser original, no obvia, y requerir análisis multivariable.""",

            "tendencia_multitemporal": f"""{data_summary}

Genera una pregunta sobre TENDENCIAS TEMPORALES O GEOGRÁFICAS que compare múltiples periodos
o regiones. La respuesta debe incluir:

- Análisis comparativo cuantitativo
- Identificación de patrones espaciales/temporales
- Factores explicativos contextuales
- Proyecciones o implicaciones

FORMATO:
P: [pregunta]
R: [respuesta con datos específicos]""",

            "impacto_socioeconomico": f"""{data_summary}

Crea una pregunta que explore el IMPACTO DE VARIABLES SOCIOECONÓMICAS en el comportamiento electoral.
La respuesta debe:

- Cuantificar relaciones usando los datos
- Discutir mecanismos causales posibles
- Considerar heterogeneidad geográfica
- Mencionar políticas públicas relevantes

FORMATO:
P: [pregunta]
R: [respuesta analítica]""",

            "auto": f"""{data_summary}

Genera la PREGUNTA MÁS INTERESANTE Y COMPLEJA que puedas formular basándote en estos datos.
Debe combinar múltiples dimensiones de análisis y requerir razonamiento sofisticado.

La respuesta debe demostrar:
1. Maestría en análisis cuantitativo
2. Conocimiento del contexto electoral español
3. Capacidad de síntesis y comunicación

FORMATO:
P: [pregunta innovadora]
R: [respuesta completa y bien estructurada]"""
        }

        return prompts.get(question_type, prompts["auto"])

    def _parse_qa_response(self, response_text: str) -> Optional[Dict]:
        """
        Parsea la respuesta de la API extrayendo pregunta y respuesta
        """
        try:
            lines = response_text.strip().split('\n')

            pregunta = None
            respuesta_lines = []
            in_respuesta = False

            for line in lines:
                line = line.strip()

                # Buscar inicio de pregunta
                if line.startswith('P:') or line.startswith('PREGUNTA:'):
                    pregunta = line.split(':', 1)[1].strip()
                    continue

                # Buscar inicio de respuesta
                if line.startswith('R:') or line.startswith('RESPUESTA:'):
                    in_respuesta = True
                    resp_text = line.split(':', 1)[1].strip()
                    if resp_text:
                        respuesta_lines.append(resp_text)
                    continue

                # Continuar respuesta
                if in_respuesta and line:
                    respuesta_lines.append(line)

            if pregunta and respuesta_lines:
                respuesta = ' '.join(respuesta_lines)

                # Validar que tenga contenido sustancial
                if len(pregunta) > 20 and len(respuesta) > 100:
                    return {
                        "instruction": pregunta,
                        "input": "Análisis complejo basado en datos electorales y socioeconómicos",
                        "output": respuesta
                    }

            # Si el parsing falla, intentar método alternativo
            return self._fallback_parse(response_text)

        except Exception as e:
            logger.error(f"Error parsing response: {e}")
            return None

    def _fallback_parse(self, text: str) -> Optional[Dict]:
        """Método alternativo de parsing si el estándar falla"""
        # Buscar cualquier patrón de pregunta-respuesta
        import re

        # Patrón: P: ... R: ...
        pattern = r'(?:P|Pregunta)[:\s]+(.+?)(?:\n\s*)(?:R|Respuesta)[:\s]+(.+)'
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)

        if match:
            pregunta = match.group(1).strip()
            respuesta = match.group(2).strip()

            return {
                "instruction": pregunta,
                "input": "Datos electorales españoles",
                "output": respuesta
            }

        return None

    def get_usage_stats(self) -> Dict:
        """Obtiene estadísticas de uso y costo"""
        return {
            "total_requests": self.requests_count,
            "total_tokens": self.total_tokens,
            "estimated_cost_usd": self.total_cost,
            "remaining_credit": 10.0 - self.total_cost,
            "avg_tokens_per_request": self.total_tokens / self.requests_count if self.requests_count > 0 else 0
        }

Esta es la clase que genera el dataset de Nivel 3, en este caso compuesto por 200 ejemplos de preguntas/respuestas complejas.

In [ ]:
class Nivel3DatasetGenerator:
    def __init__(self, df: pd.DataFrame, api_key: str):
        """
        Generador completo de dataset Nivel 3

        Args:
            df: DataFrame completo con todos los datos
            api_key: API Key de DeepSeek
        """
        self.df = df
        self.api_generator = DeepSeekAPIGenerator(api_key)

        # Cache para no repetir muestras
        self.generated_samples = set()

    def identify_interesting_cases(self, n_cases: int = 50) -> List[pd.DataFrame]:
        """
        Identifica casos interesantes para análisis complejo

        Args:
            n_cases: Número de casos a identificar

        Returns:
            Lista de DataFrames con muestras interesantes
        """
        interesting_cases = []

        # Estrategia 1: Municipios con alta variabilidad electoral
        if 'Municipio' in self.df.columns:
            municipio_stats = self.df.groupby('Municipio').agg({
                '% PP': ['std', 'mean'],
                '% PSOE': ['std', 'mean'],
                'Participación': 'std'
            }).round(2)

            # Municipios con alta variabilidad interna
            high_variability = municipio_stats[('% PP', 'std')].nlargest(20).index.tolist()

            for mun in high_variability[:10]:
                sample = self.df[self.df['Municipio'] == mun]
                if len(sample) >= 5:  # Al menos 5 secciones
                    interesting_cases.append(sample)

        # Estrategia 2: Muestras por tipo de municipio
        if 'Renta persona 2017' in self.df.columns:
            # Municipios por quintil de renta
            renta_bins = pd.qcut(self.df['Renta persona 2017'], q=5, duplicates='drop')

            for bin_label in renta_bins.unique():
                if pd.notna(bin_label):
                    sample = self.df[renta_bins == bin_label].sample(min(100, len(self.df[renta_bins == bin_label])), random_state=42)
                    interesting_cases.append(sample)

        # Estrategia 3: Combinaciones geográficas
        if 'Provincia' in self.df.columns:
            provincias_interesantes = ['Madrid', 'Barcelona', 'Sevilla', 'Valencia', 'Bilbao']

            for prov in provincias_interesantes:
                if prov in self.df['Provincia'].values:
                    sample = self.df[self.df['Provincia'] == prov].sample(
                        min(50, len(self.df[self.df['Provincia'] == prov])),
                        random_state=42
                    )
                    interesting_cases.append(sample)

        # Estrategia 4: Casos extremos
        extremos = []

        for col in ['% PP', '% PSOE', 'Participación', 'Renta persona 2017']:
            if col in self.df.columns:
                # Valores más altos
                high_idx = self.df[col].nlargest(5).index
                extremos.extend(high_idx.tolist())

                # Valores más bajos
                low_idx = self.df[col].nsmallest(5).index
                extremos.extend(low_idx.tolist())

        if extremos:
            sample = self.df.loc[extremos[:20]]
            interesting_cases.append(sample)

        # Mezclar y limitar
        random.shuffle(interesting_cases)
        return interesting_cases[:n_cases]

    def generate_dataset(self,
                        n_examples: int = 200,
                        batch_size: int = 10,
                        output_path: str = "dataset_nivel3.jsonl",
                        checkpoint_freq: int = 20) -> List[Dict]:
        """
        Genera el dataset completo de nivel 3

        Args:
            n_examples: Número total de ejemplos a generar
            batch_size: Ejemplos por lote
            output_path: Ruta para guardar resultados
            checkpoint_freq: Frecuencia de guardado automático

        Returns:
            Lista con todos los ejemplos generados
        """
        logger.info(f"Iniciando generación de {n_examples} ejemplos nivel 3")

        # Identificar casos interesantes
        interesting_cases = self.identify_interesting_cases(n_cases=min(100, n_examples * 2))
        logger.info(f"Identificados {len(interesting_cases)} casos interesantes")

        # Tipos de preguntas a generar
        question_types = [
            "correlacion_avanzada",
            "tendencia_multitemporal",
            "impacto_socioeconomico",
            "auto"
        ]

        all_examples = []
        examples_generated = 0
        batch_count = 0

        # Cargar progreso previo si existe
        if os.path.exists(output_path):
            with open(output_path, 'r') as f:
                for line in f:
                    if line.strip():
                        all_examples.append(json.loads(line))
                        examples_generated += 1
            logger.info(f"Cargados {examples_generated} ejemplos previos")

        # Generar hasta alcanzar el objetivo
        while examples_generated < n_examples:
            batch_count += 1
            logger.info(f"--- Procesando lote {batch_count} ---")

            print(examples_generated)

            if examples_generated % 5 == 0:
              print(f"Ejemplos generados: {examples_generated}")

            batch_examples = []

            # Preparar muestras para este batch
            batch_samples = []
            for _ in range(batch_size):
                if interesting_cases:
                    sample = random.choice(interesting_cases)
                    batch_samples.append(sample)
                else:
                    # Si se acaban los casos interesantes, muestrear aleatoriamente
                    sample = self.df.sample(min(20, len(self.df)), random_state=examples_generated)
                    batch_samples.append(sample)

            # Generar en paralelo (threads para I/O bound)
            with ThreadPoolExecutor(max_workers=min(5, batch_size)) as executor:
                future_to_sample = {}

                for i, sample in enumerate(batch_samples):
                    q_type = random.choice(question_types)
                    future = executor.submit(
                        self.api_generator.generate_complex_qa,
                        sample,
                        sample_size=min(10, len(sample)),
                        question_type=q_type
                    )
                    future_to_sample[future] = (i, q_type)

                # Procesar resultados
                for future in as_completed(future_to_sample):
                    i, q_type = future_to_sample[future]

                    try:
                        result = future.result(timeout=120)

                        if result:
                            # Añadir información del sample
                            sample_info = {
                                "tamano_muestra": len(batch_samples[i]),
                                "columnas_disponibles": list(batch_samples[i].columns),
                                "tipo_pregunta": q_type
                            }
                            result["metadata"].update(sample_info)

                            batch_examples.append(result)
                            examples_generated += 1

                            logger.info(f"✅ Generado ejemplo {examples_generated}/{n_examples}")

                            # Verificar crédito restante
                            stats = self.api_generator.get_usage_stats()
                            if stats["remaining_credit"] < 0.5:
                                logger.warning(f"⚠️  Crédito bajo: ${stats['remaining_credit']:.2f}")
                                return all_examples

                    except Exception as e:
                        logger.error(f"Error en generación: {str(e)}")

            # Añadir ejemplos del batch
            all_examples.extend(batch_examples)

            # Guardar checkpoint
            if batch_count % checkpoint_freq == 0 or examples_generated >= n_examples:
                self._save_checkpoint(all_examples, output_path)

                stats = self.api_generator.get_usage_stats()
                logger.info(f"Checkpoint guardado. Ejemplos: {examples_generated}")
                logger.info(f"Estadísticas API: {stats}")

            # Pausa entre batches para evitar rate limiting
            if examples_generated < n_examples:
                time.sleep(2)

        # Guardar final
        self._save_checkpoint(all_examples, output_path)

        # Estadísticas finales
        stats = self.api_generator.get_usage_stats()
        logger.info("=" * 50)
        logger.info("GENERACIÓN COMPLETADA")
        logger.info(f"Total ejemplos: {len(all_examples)}")
        logger.info(f"Total requests API: {stats['total_requests']}")
        logger.info(f"Total tokens: {stats['total_tokens']:,}")
        logger.info(f"Costo estimado: ${stats['estimated_cost_usd']:.4f}")
        logger.info(f"Crédito restante: ${stats['remaining_credit']:.2f}")
        logger.info("=" * 50)

        return all_examples

    def _save_checkpoint(self, examples: List[Dict], output_path: str):
        """Guarda checkpoint del dataset"""
        try:
            # Guardar en JSONL
            with open(output_path, 'w', encoding='utf-8') as f:
                for example in examples:
                    f.write(json.dumps(example, ensure_ascii=False) + '\n')

            # También guardar backup con timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_path = f"{output_path.rsplit('.', 1)[0]}_backup_{timestamp}.jsonl"
            with open(backup_path, 'w', encoding='utf-8') as f:
                for example in examples:
                    f.write(json.dumps(example, ensure_ascii=False) + '\n')

            logger.info(f"Checkpoint guardado en {output_path} (backup: {backup_path})")

        except Exception as e:
            logger.error(f"Error guardando checkpoint: {e}")


Esta es la función main() que define el tamaño de dataset, almacena los path del dataset original, y va mostrando el progreso de la generación del dataset.

In [ ]:

# ============================================================================
# SCRIPT PRINCIPAL DE EJECUCIÓN
# ============================================================================

def main():
    """Función principal de ejecución"""

    # CONFIGURACIÓN
    API_KEY = "he_eliminado_mi_clave_por_seguridad"  # ¡REEMPLAZAR!
    CSV_PATH = "/content/drive/MyDrive/Practica_LLM_Engineering_25/df_elecciones_19.csv"  # Ruta a tu dataset combinado
    OUTPUT_PATH = "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_nivel3.jsonl"
    NUM_EJEMPLOS = 200  # Objetivo de ejemplos nivel 3
    BATCH_SIZE = 8  # Ejemplos por lote

    print("=" * 60)
    print("GENERADOR NIVEL 3 - PREGUNTAS COMPLEJAS")
    print("=" * 60)

    # 1. Cargar datos
    print("1. Cargando dataset...")
    try:
        df = pd.read_csv(CSV_PATH)
        print(f"   ✓ Dataset cargado: {len(df)} filas, {len(df.columns)} columnas")
    except Exception as e:
        print(f"   ✗ Error cargando dataset: {e}")
        return

    # 2. Verificar columnas esenciales
    essential_cols = ['Municipio', '% PP', '% PSOE', 'Participación']
    missing_cols = [col for col in essential_cols if col not in df.columns]

    if missing_cols:
        print(f"   ⚠️  Columnas faltantes: {missing_cols}")
        print("   Continuando con columnas disponibles...")

    # 3. Inicializar generador
    print("2. Inicializando generador con DeepSeek API...")
    try:
        generator = Nivel3DatasetGenerator(df, API_KEY)
        print("   ✓ Generador inicializado")
    except Exception as e:
        print(f"   ✗ Error inicializando generador: {e}")
        return

    # 4. Generar dataset
    print(f"3. Generando {NUM_EJEMPLOS} ejemplos nivel 3...")
    print("   Esto puede tomar varios minutos y consumir crédito de API")
    print("   Presiona Ctrl+C para interrumpir y guardar progreso")
    print("-" * 60)

    try:
        dataset = generator.generate_dataset(
            n_examples=NUM_EJEMPLOS,
            batch_size=BATCH_SIZE,
            output_path=OUTPUT_PATH
        )

        print("\n" + "=" * 60)
        print("🎉 GENERACIÓN COMPLETADA EXITOSAMENTE")
        print("=" * 60)

        # Mostrar algunos ejemplos
        print("\nEJEMPLOS GENERADOS:")
        for i, example in enumerate(dataset[:3]):
            print(f"\n--- Ejemplo {i+1} ---")
            print(f"Pregunta: {example['instruction'][:100]}...")
            print(f"Respuesta: {example['output'][:150]}...")
            print(f"Tipo: {example['metadata'].get('tipo_pregunta', 'N/A')}")
            print(f"Tokens: {example['metadata'].get('tokens_utilizados', 'N/A')}")

        # Estadísticas finales
        stats = generator.api_generator.get_usage_stats()
        print(f"\n📊 ESTADÍSTICAS FINALES:")
        print(f"   Ejemplos generados: {len(dataset)}")
        print(f"   Requests API: {stats['total_requests']}")
        print(f"   Tokens totales: {stats['total_tokens']:,}")
        print(f"   Costo estimado: ${stats['estimated_cost_usd']:.4f}")
        print(f"   Crédito restante: ${stats['remaining_credit']:.2f}")
        print(f"\n💾 Dataset guardado en: {OUTPUT_PATH}")

    except KeyboardInterrupt:
        print("\n\n⚠️  Generación interrumpida por usuario")
        print("Progreso guardado en archivo de salida")
    except Exception as e:
        print(f"\n✗ Error durante la generación: {e}")
        import traceback
        traceback.print_exc()


Activamos la generación del dataset.

In [ ]:
if __name__ == "__main__":
    # Para ejecutar localmente
    main()


GENERADOR NIVEL 3 - PREGUNTAS COMPLEJAS
1. Cargando dataset...
   ✓ Dataset cargado: 72619 filas, 97 columnas
2. Inicializando generador con DeepSeek API...
   ✓ Generador inicializado
3. Generando 200 ejemplos nivel 3...
   Esto puede tomar varios minutos y consumir crédito de API
   Presiona Ctrl+C para interrumpir y guardar progreso
------------------------------------------------------------
24
32
40
Ejemplos generados: 40
48
56
64
72
80
Ejemplos generados: 80
88
96
104
112
120
Ejemplos generados: 120
128
136
144
152
160
Ejemplos generados: 160
168
176
184
192

🎉 GENERACIÓN COMPLETADA EXITOSAMENTE

EJEMPLOS GENERADOS:

--- Ejemplo 1 ---
Pregunta: Considerando los datos electorales y socioeconómicos de la muestra, analice las tendencias geográfic...
Respuesta: El análisis de los datos revela tendencias geográficas y socioeconómicas diferenciadas. En términos de apoyo electoral, el PSOE presenta una media más...
Tipo: tendencia_multitemporal
Tokens: 2122

--- Ejemplo 2 ---
Pregunta: 